In [1]:
# =============================================
# ÉTAPE 1 : IMPORTS ET CONFIGURATION
# =============================================
# EXÉCUTEZ CETTE CELLULE EN PREMIER

import os
import time
import base64

# Configuration
SOURCE_DIR = r"C:\Users\jcbouquin\AI\GenAI\company_process\Depots\Sel_Files"

print("[OK] Configuration chargée")
print(f"[SOURCE] {SOURCE_DIR}")

# Vérification Selenium
try:
    import selenium
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    print("[OK] Selenium disponible")
except ImportError:
    print("[ERREUR] Installez Selenium : pip install selenium webdriver-manager")


[OK] Configuration chargée
[SOURCE] C:\Users\jcbouquin\AI\GenAI\company_process\Depots\Sel_Files
[OK] Selenium disponible


In [2]:
# =============================================
# ÉTAPE 2 : FONCTIONS PRINCIPALES  
# =============================================
# EXÉCUTEZ APRÈS LA CELLULE 1

def creer_lots_simple(source_dir, taille_max_mo=65):
    """Crée des lots de fichiers de 65 Mo maximum"""
    
    print(f"[ANALYSE] Dossier : {source_dir}")
    
    if not os.path.exists(source_dir):
        raise Exception(f"Dossier introuvable : {source_dir}")
    
    # Analyser les fichiers
    debut = time.time()
    tous_fichiers = []
    
    for filename in os.listdir(source_dir):
        filepath = os.path.join(source_dir, filename)
        if os.path.isfile(filepath):
            taille_bytes = os.path.getsize(filepath)
            taille_mo = taille_bytes / (1024 * 1024)
            
            tous_fichiers.append({
                'name': filename,
                'path': filepath, 
                'size_bytes': taille_bytes,
                'size_mo': taille_mo
            })
    
    temps = (time.time() - debut) * 1000
    taille_totale = sum(f['size_mo'] for f in tous_fichiers)
    
    print(f"[OK] {len(tous_fichiers)} fichiers analysés en {temps:.0f}ms")
    print(f"[INFO] Taille totale : {taille_totale:.1f} Mo")
    
    # Créer les lots
    lots = []
    lot_courant = []
    taille_lot = 0
    
    for fichier in tous_fichiers:
        if fichier['size_mo'] > taille_max_mo:
            # Fichier trop volumineux - lot séparé
            if lot_courant:
                lots.append(lot_courant)
                lot_courant = []
                taille_lot = 0
            lots.append([fichier])
            print(f"[WARNING] Fichier volumineux : {fichier['name']} ({fichier['size_mo']:.1f} Mo)")
        elif taille_lot + fichier['size_mo'] > taille_max_mo:
            # Nouveau lot nécessaire
            if lot_courant:
                lots.append(lot_courant)
            lot_courant = [fichier]
            taille_lot = fichier['size_mo']
        else:
            # Ajouter au lot courant
            lot_courant.append(fichier)
            taille_lot += fichier['size_mo']
    
    # Finaliser
    if lot_courant:
        lots.append(lot_courant)
    
    # Résumé
    print(f"\\n[SUCCESS] {len(lots)} lots créés :")
    for i, lot in enumerate(lots, 1):
        taille = sum(f['size_mo'] for f in lot)
        print(f"  Lot {i} : {len(lot)} fichiers ({taille:.1f} Mo)")
    
    return lots

def upload_complet():
    """Fonction principale d'upload"""
    
    print("="*50)
    print("[UPLOAD] Processus complet par lots")
    print("="*50)
    
    try:
        # Créer les lots
        lots = creer_lots_simple(SOURCE_DIR, 65)
        
        if not lots:
            print("[ERREUR] Aucun fichier trouvé")
            return
        
        print(f"\\n[INFO] {len(lots)} lots prêts à uploader")
        print("[INFO] Chaque lot fait maximum 65 Mo")
        
        # Ouvrir SharePoint
        print("\\n[SHAREPOINT] Ouverture...")
        url = "https://quintiles.sharepoint.com/sites/Orion_GenAI/Shared%20Documents/Forms/AllItems.aspx?e=5%3A580acf7363564cfc9fe468be6e34250a&sharingv2=true&fromShare=true&at=9&CID=fc12a4d8%2D4995%2D4d0b%2D8e2a%2Df69209919598&FolderCTID=0x01200059A44C4A87A34D449D25609A880D0432&id=%2Fsites%2FOrion%5FGenAI%2FShared%20Documents%2Fapps%2Fsearchsphere%2FFrance%2Fraw"
        
        options = Options()
        options.add_argument('--start-maximized')
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(5)
        
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        print("[OK] SharePoint ouvert")
        
        # Upload chaque lot
        lots_reussis = 0
        
        for i, lot in enumerate(lots, 1):
            print(f"\\n[UPLOAD] Lot {i}/{len(lots)} - {len(lot)} fichiers")
            
            try:
                # Préparer les fichiers
                files_data = []
                for fichier in lot:
                    with open(fichier['path'], 'rb') as f:
                        content = base64.b64encode(f.read()).decode('utf-8')
                        files_data.append({
                            'name': fichier['name'],
                            'content': content,
                            'size': fichier['size_bytes'],
                            'type': 'application/octet-stream'
                        })
                
                # JavaScript drag & drop
                js_script = '''
        // Fonction pour convertir base64 en Blob
        function base64ToBlob(base64, mimeType) {
            const byteCharacters = atob(base64);
            const byteNumbers = new Array(byteCharacters.length);
            for (let i = 0; i < byteCharacters.length; i++) {
                byteNumbers[i] = byteCharacters.charCodeAt(i);
            }
            const byteArray = new Uint8Array(byteNumbers);
            return new Blob([byteArray], {type: mimeType});
        }
        
        // Fonction pour créer un fichier File
        function createFile(name, base64Content, mimeType) {
            const blob = base64ToBlob(base64Content, mimeType);
            return new File([blob], name, {type: mimeType});
        }
        
        // Créer les fichiers
        const files = arguments[0];  // Array des fichiers passé depuis Python
        const fileObjects = files.map(f => createFile(f.name, f.content, f.type));
        
        // Trouver la zone de drop
        const dropZones = [
            document.querySelector('div[data-automationid="list-content"][role="grid"]'),
            document.querySelector('div[data-automationid="list-empty-placeholder"]'),
            document.querySelector('div[data-automationid="portalhost-placeholder"]'),
            document.querySelector('.od-ItemContent-item'),
            document.querySelector('[role="grid"]')
        ];
        
        let dropZone = null;
        for (let zone of dropZones) {
            if (zone && zone.offsetParent !== null) {
                dropZone = zone;
                break;
            }
        }
        
        if (!dropZone) {
            throw new Error("Zone de drop introuvable");
        }
        
        console.log("Zone de drop trouvée:", dropZone);
        
        // Créer et déclencher les événements de drag & drop
        const dataTransfer = new DataTransfer();
        fileObjects.forEach(file => {
            dataTransfer.items.add(file);
        });
        
        // Événement dragenter
        const dragenterEvent = new DragEvent('dragenter', {
            bubbles: true,
            cancelable: true,
            dataTransfer: dataTransfer
        });
        
        // Événement dragover
        const dragoverEvent = new DragEvent('dragover', {
            bubbles: true,
            cancelable: true,
            dataTransfer: dataTransfer
        });
        
        // Événement drop
        const dropEvent = new DragEvent('drop', {
            bubbles: true,
            cancelable: true,
            dataTransfer: dataTransfer
        });
        
        // Simuler la séquence complète
        dropZone.dispatchEvent(dragenterEvent);
        setTimeout(() => {
            dropZone.dispatchEvent(dragoverEvent);
            setTimeout(() => {
                dropZone.dispatchEvent(dropEvent);
                console.log("Drop simulé avec", fileObjects.length, "fichiers");
            }, 100);
        }, 100);
        
        return "Drop simulé avec succès";
        '''
                
                result = driver.execute_script(js_script, files_data)
                print(f"[OK] {result}")
                
                # Attendre
                duree = min(15, max(5, len(files_data) * 2))
                print(f"[WAIT] {duree}s...")
                time.sleep(duree)
                
                lots_reussis += 1
                print(f"[SUCCESS] Lot {i} uploadé !")
                
                # Pause entre lots
                if i < len(lots):
                    time.sleep(3)
                    
            except Exception as e:
                print(f"[ERREUR] Lot {i} : {e}")
        
        # Bilan
        print(f"\\n{'='*50}")
        print("[BILAN FINAL]")
        print(f"{'='*50}")
        
        if lots_reussis == len(lots):
            print("[SUCCESS] TOUS LES LOTS UPLOADÉS !")
            total_fichiers = sum(len(lot) for lot in lots)
            print(f"[RESULT] {total_fichiers} fichiers transférés")
        else:
            print(f"[PARTIAL] {lots_reussis}/{len(lots)} lots réussis")
        
        print("[INFO] Vérifiez SharePoint !")
        
    except Exception as e:
        print(f"[ERREUR] {e}")
    
    finally:
        try:
            driver.quit()
            print("\\n[OK] Session fermée")
        except:
            pass

print("[OK] Fonctions prêtes !")


[OK] Fonctions prêtes !


In [3]:
# =============================================
# ÉTAPE 3 : TEST (SANS UPLOAD)
# =============================================
# EXÉCUTEZ CETTE CELLULE POUR TESTER SEULEMENT

print("="*50)
print("[TEST] Création des lots - SANS UPLOAD")
print("="*50)

try:
    lots = creer_lots_simple(SOURCE_DIR, 65)
    
    if lots:
        total_fichiers = sum(len(lot) for lot in lots)
        total_taille = sum(sum(f['size_mo'] for f in lot) for lot in lots)
        
        print(f"\n[RESULTAT TEST]")
        print(f"  Fichiers   : {total_fichiers}")
        print(f"  Taille     : {total_taille:.1f} Mo")
        print(f"  Lots créés : {len(lots)}")
        print(f"\n[OK] Test réussi ! Vous pouvez lancer l'upload.")
    else:
        print("[ERREUR] Aucun fichier trouvé")
        
except Exception as e:
    print(f"[ERREUR] {e}")
    print("[INFO] Vérifiez que le dossier Sel_Files existe et contient des fichiers")


[TEST] Création des lots - SANS UPLOAD
[ANALYSE] Dossier : C:\Users\jcbouquin\AI\GenAI\company_process\Depots\Sel_Files
[OK] 19 fichiers analysés en 2ms
[INFO] Taille totale : 437.0 Mo
\n[SUCCESS] 9 lots créés :
  Lot 1 : 4 fichiers (62.0 Mo)
  Lot 2 : 3 fichiers (57.3 Mo)
  Lot 3 : 3 fichiers (46.7 Mo)
  Lot 4 : 3 fichiers (54.6 Mo)
  Lot 5 : 1 fichiers (23.3 Mo)
  Lot 6 : 1 fichiers (58.5 Mo)
  Lot 7 : 1 fichiers (61.4 Mo)
  Lot 8 : 2 fichiers (52.9 Mo)
  Lot 9 : 1 fichiers (20.4 Mo)

[RESULTAT TEST]
  Fichiers   : 19
  Taille     : 437.0 Mo
  Lots créés : 9

[OK] Test réussi ! Vous pouvez lancer l'upload.


In [5]:
# =============================================
# ÉTAPE 4 : UPLOAD COMPLET
# =============================================
# EXÉCUTEZ CETTE CELLULE POUR L'UPLOAD VERS SHAREPOINT

print("="*50)
print("[UPLOAD] Lancement du processus complet")
print("="*50)
print("[WARNING] Ceci va uploader TOUS vos fichiers !")
print("[INFO] Assurez-vous d'avoir fermé SharePoint dans votre navigateur")
print()

# Lancement de l'upload complet
upload_complet()


[UPLOAD] Lancement du processus complet
[WARNING] Ceci va uploader TOUS vos fichiers !
[INFO] Assurez-vous d'avoir fermé SharePoint dans votre navigateur

[UPLOAD] Processus complet par lots
[ANALYSE] Dossier : C:\Users\jcbouquin\AI\GenAI\company_process\Depots\Sel_Files
[OK] 19 fichiers analysés en 2ms
[INFO] Taille totale : 437.0 Mo
\n[SUCCESS] 9 lots créés :
  Lot 1 : 4 fichiers (62.0 Mo)
  Lot 2 : 3 fichiers (57.3 Mo)
  Lot 3 : 3 fichiers (46.7 Mo)
  Lot 4 : 3 fichiers (54.6 Mo)
  Lot 5 : 1 fichiers (23.3 Mo)
  Lot 6 : 1 fichiers (58.5 Mo)
  Lot 7 : 1 fichiers (61.4 Mo)
  Lot 8 : 2 fichiers (52.9 Mo)
  Lot 9 : 1 fichiers (20.4 Mo)
\n[INFO] 9 lots prêts à uploader
[INFO] Chaque lot fait maximum 65 Mo
\n[SHAREPOINT] Ouverture...
[OK] SharePoint ouvert
\n[UPLOAD] Lot 1/9 - 4 fichiers
[OK] Drop simulé avec succès
[WAIT] 8s...
[SUCCESS] Lot 1 uploadé !
\n[UPLOAD] Lot 2/9 - 3 fichiers
[OK] Drop simulé avec succès
[WAIT] 6s...
[SUCCESS] Lot 2 uploadé !
\n[UPLOAD] Lot 3/9 - 3 fichiers
[OK]